In [ ]:
# Import
import json
import random
import numpy as np
import rasterio as rio
from pathlib import Path
from math import floor, ceil
from itertools import product
from functools import partial
from keras.models import Model
from keras.utils import Sequence
from keras.optimizers import Adam
from keras.regularizers import l2
# from keras.layers import LeakyReLU
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from rasterio import windows as rio_windows
from keras.utils.vis_utils import plot_model
from keras.layers import Input, Dense, Dropout
from keras.layers import UpSampling2D, concatenate
from keras.layers import Conv2D, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D

In [ ]:
# Define Parameters
droprate = 0.1
image_height = 6000
image_width = 6000
window_height = 512
window_width = 512
min_height_overlap = 64
min_width_overlap = 64
boundless_flag = True
class_count = 6
data_shuffle = True
batchsize=2
bands = (1, 2, 3, 4, 5)
image_features = len(bands)

In [ ]:
geo_augs = (
    (lambda m : m),
    partial(np.rot90, k=1, axes=(1, 2)),
    partial(np.rot90, k=2, axes=(1, 2)),
    partial(np.rot90, k=3, axes=(1, 2)),
    partial(np.flip, axis=1),
    partial(np.flip, axis=2)
)

In [ ]:
# Input Data
config_dir = Path('Configs')
train_config = config_dir / 'Train_Map.json'
valid_config = config_dir / 'Validation_Map.json'
test_config = config_dir / 'Test_Map.json'
model_dir = Path("Models")
mplot = model_dir / "Model_Plot.png"
model_max_accuracy = model_dir / 'Model_MaxAccuracy.h5' 
model_min_loss = model_dir / 'Model_MinLoss.h5'
log_d = Path('Logs')

with open(train_config.as_posix(), 'r') as tm:
    train_map = json.load(tm)

with open(valid_config.as_posix(), 'r') as tm:
    valid_map = json.load(tm)

In [ ]:
def generate_windows(img_height, img_width, win_height, win_width, min_hoverlap, min_woverlap, boundless=False):
    hc = ceil((img_height - min_hoverlap) / (win_height - min_hoverlap))
    wc = ceil((img_width - min_woverlap) / (win_width - min_woverlap))
    
    
    h_overlap = ((hc * win_height) - img_height) // (hc - 1)
    w_overlap = ((wc * win_height) - img_width) // (wc - 1)
    
    
    hslack_res = ((hc * win_height) - img_height) % (hc - 1)
    wslack_res = ((wc * win_width) - img_width) % (wc - 1)
    
    dh = win_height - h_overlap
    dw = win_width - w_overlap
    
    row_offsets = np.arange(0, (img_height-h_overlap), dh)
    col_offsets = np.arange(0, (img_width-w_overlap), dw)
    
    if hslack_res > 0:
        row_offsets[-hslack_res:] -= np.arange(1, (hslack_res + 1), 1)
    if wslack_res > 0:
        col_offsets[-wslack_res:] -= np.arange(1, (wslack_res + 1), 1)
    
    row_offsets = row_offsets.tolist()
    col_offsets = col_offsets.tolist()
    
    offsets = product(col_offsets, row_offsets)
    
    indices = product(range(len(col_offsets)), range(len(row_offsets)))
    
    big_window = rio_windows.Window(col_off=0, row_off=0, width=img_width, height=img_height)
    
    for index, (col_off, row_off) in zip(indices, offsets):
        window = rio_windows.Window(
            col_off=col_off,
            row_off=row_off,
            width=win_width,
            height=win_height
        )
        if boundless:
            yield index, window
        else:
            yield index, window.intersection(big_window)

In [ ]:
class RasterDataGenerator(Sequence):
    def __init__(
        self,  
        map_dict,
        channels,
        img_height,
        img_width,
        win_height,
        win_width,
        min_hoverlap,
        min_woverlap,
        cls_count,
        boundless=False,
        shuffle=True,
        batch_size=1,
    ):
        assert isinstance(map_dict, dict), 'Invalid type for parameter <map_dict>, expected type `dict`!'
        assert all([set(map_dict[k].keys()) == {'IMAGE', 'LABEL'} for k in map_dict.keys()]), "Invalid map <dict_map>, Key Mismatch!"
        
        couples =  [(Path(couple['IMAGE']).as_posix(), Path(couple['LABEL']).as_posix()) for couple in map_dict.values()]
        
        windows = list(
            generate_windows(
                img_height=img_height,
                img_width=img_width,
                win_height=win_height,
                win_width=win_width,
                min_hoverlap=min_hoverlap,
                min_woverlap=min_woverlap,
                boundless=boundless
            )
        )
        dat = list(product(couples, windows))
        if shuffle:
            random.shuffle(dat)
        self.data = dat
        self.channels = channels
        self.class_count = cls_count
        self.batch_size = batch_size
    
    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        current_batch = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]
        islices = list()
        lslices = list()
        for (im, lb), (_, w) in current_batch:
            with rio.open(im, 'r') as isrc:
                islice = isrc.read(indexes=self.channels, window=w, boundless=boundless_flag, masked=False)
                islice = np.moveaxis(a=islice, source=0, destination=-1)
                islices.append(islice / 255.0)
            with rio.open(lb, 'r') as lsrc:
                lslice = lsrc.read(window=w, boundless=boundless_flag, masked=False)
                lslice = np.moveaxis(a=lslice, source=0, destination=-1)
                lslice =to_categorical(
                    y=(lslice-1), 
                    num_classes=self.class_count
                )
                lslices.append(lslice)
        ibatch = np.stack(islices, axis=0)
        lbatch = np.stack(lslices, axis=0)
        return ibatch, lbatch

In [ ]:
# Begin
input_layer = Input(
    shape=(None, None, image_features), 
    name='input_layer'
)

# Part e1
convolve_layer_1a = Conv2D(
    filters=64, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl1a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_1a'
)(input_layer)

norm_layer_1a = BatchNormalization(name='norm_layer_1a')(convolve_layer_1a)

convolve_layer_1b = Conv2D(
    filters=64, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl1b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_1b'
)(norm_layer_1a)

norm_layer_1b = BatchNormalization(name='norm_layer_1b')(convolve_layer_1b)

drop_layer_1 = Dropout(droprate, name='drop_layer_1')(norm_layer_1b)

pooling_layer_1 = MaxPooling2D(
    pool_size=(2, 2),
    padding='same',
    data_format='channels_last',
    name='pooling_layer_1'
)(drop_layer_1)


# Part e2
convolve_layer_2a = Conv2D(
    filters=128, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl2a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_2a'
)(pooling_layer_1)

norm_layer_2a = BatchNormalization(name='norm_layer_2a')(convolve_layer_2a)

convolve_layer_2b = Conv2D(
    filters=128, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl2b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_2b'
)(norm_layer_2a)

norm_layer_2b = BatchNormalization(name='norm_layer_2b')(convolve_layer_2b)

drop_layer_2 = Dropout(droprate, name='drop_layer_2')(norm_layer_2b)

pooling_layer_2 = MaxPooling2D(
    pool_size=(2, 2),
    padding='same',
    data_format='channels_last',
    name='pooling_layer_2'
)(drop_layer_2)


# Part e3
convolve_layer_3a = Conv2D(
    filters=256, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl3a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_3a'
)(pooling_layer_2)

norm_layer_3a = BatchNormalization(name='norm_layer_3a')(convolve_layer_3a)

convolve_layer_3b = Conv2D(
    filters=256, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl3b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_3b'
)(norm_layer_3a)

norm_layer_3b = BatchNormalization(name='norm_layer_3b')(convolve_layer_3b)

drop_layer_3 = Dropout(droprate, name='drop_layer_3')(norm_layer_3b)

pooling_layer_3 = MaxPooling2D(
    pool_size=(2, 2),
    padding='same',
    data_format='channels_last',
    name='pooling_layer_3'
)(drop_layer_3)

# Part e4
convolve_layer_4a = Conv2D(
    filters=512, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl4a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_4a'
)(pooling_layer_3)

norm_layer_4a = BatchNormalization(name='norm_layer_4a')(convolve_layer_4a)

convolve_layer_4b = Conv2D(
    filters=512, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl4b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    # kernel_initializer='glorot_uniform',
    name='convolve_layer_4b'
)(norm_layer_4a)

norm_layer_4b = BatchNormalization(name='norm_layer_4b')(convolve_layer_4b)

drop_layer_4 = Dropout(droprate, name='drop_layer_4')(norm_layer_4b)

pooling_layer_4 = MaxPooling2D(
    pool_size=(2, 2),
    padding='same',
    data_format='channels_last',
    name='pooling_layer_4'
)(drop_layer_4)

# Part Center
convolve_layer_xa = Conv2D(
    filters=1024, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_clxa'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_xa'
)(pooling_layer_4)

norm_layer_xa = BatchNormalization(name='norm_layer_xa')(convolve_layer_xa)

convolve_layer_xb = Conv2D(
    filters=1024, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_clxb'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_xb'
)(norm_layer_xa)

norm_layer_xb = BatchNormalization(name='norm_layer_xb')(convolve_layer_xb)

drop_layer_x = Dropout(droprate, name='drop_layer_x')(norm_layer_xb)


# Part d1
upsample_layer_1 = Conv2D(
    filters=512, 
    kernel_size=(2, 2), 
#     activation=LeakyReLU(alpha=0.1, name='act_ul1'), 
    activation='relu',
    padding='same',
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='upsample_layer_1'
)(
    UpSampling2D(
        size=(2, 2),
        data_format='channels_last',
    )(drop_layer_x)
)

merge_layer_1 = concatenate(
    [
        drop_layer_4, 
        upsample_layer_1
    ],
    axis = 3, 
    name='merge_layer_1'
)

convolve_layer_5a = convolve_layer = Conv2D(
    filters=512, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl5a'),
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_5a'
)(merge_layer_1)

convolve_layer_5b = convolve_layer = Conv2D(
    filters=512, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl5b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_5b'
)(convolve_layer_5a)

# Part d2
upsample_layer_2 = Conv2D(
    filters=256, 
    kernel_size=(2, 2), 
#     activation=LeakyReLU(alpha=0.1, name='act_ul2'),
    activation='relu',
    padding='same',
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='upsample_layer_2'
)(
    UpSampling2D(
        size=(2, 2),
        data_format='channels_last',
    )(convolve_layer_5b)
)

merge_layer_2 = concatenate(
    [
        drop_layer_3, 
        upsample_layer_2
    ],
    axis = 3, 
    name='merge_layer_2'
)

convolve_layer_6a = convolve_layer = Conv2D(
    filters=256, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl6a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_6a'
)(merge_layer_2)

convolve_layer_6b = convolve_layer = Conv2D(
    filters=256, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl6b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_6b'
)(convolve_layer_6a)

# Part d3
upsample_layer_3 = Conv2D(
    filters=128, 
    kernel_size=(2, 2), 
#     activation=LeakyReLU(alpha=0.1, name='act_ul3'), 
    activation='relu',
    padding='same',
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='upsample_layer_3'
)(
    UpSampling2D(
        size=(2, 2),
        data_format='channels_last',
    )(convolve_layer_6b)
)

merge_layer_3 = concatenate(
    [
        drop_layer_2, 
        upsample_layer_3
    ],
    axis = 3, 
    name='merge_layer_3'
)

convolve_layer_7a = convolve_layer = Conv2D(
    filters=128, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl7a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_7a'
)(merge_layer_3)

convolve_layer_7b = convolve_layer = Conv2D(
    filters=128, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl5b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_7b'
)(convolve_layer_7a)

# Part d4
upsample_layer_4 = Conv2D(
    filters=64, 
    kernel_size=(2, 2), 
#     activation=LeakyReLU(alpha=0.1, name='act_ul4'), 
    activation='relu',
    padding='same',
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='upsample_layer_4'
)(
    UpSampling2D(
        size=(2, 2),
        data_format='channels_last',
    )(convolve_layer_7b)
)

merge_layer_4 = concatenate(
    [
        drop_layer_1, 
        upsample_layer_4
    ],
    axis = 3, 
    name='merge_layer_4'
)

convolve_layer_8a = convolve_layer = Conv2D(
    filters=64, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl8a'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_8a'
)(merge_layer_4)

convolve_layer_8b = convolve_layer = Conv2D(
    filters=64, 
    kernel_size=(3, 3), 
#     activation=LeakyReLU(alpha=0.1, name='act_cl8b'), 
    activation='relu',
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='convolve_layer_8b'
)(convolve_layer_8a)

output_layer = Conv2D(
    filters=class_count, 
    kernel_size=(1, 1), 
    activation='softmax', 
    padding='same', 
    data_format='channels_last',
    dilation_rate=1,
    # kernel_regularizer=l2(0.01), 
    # bias_regularizer=l2(0.01),
    kernel_initializer='glorot_uniform',
    name='output_layer'
)(convolve_layer_8b)

In [ ]:
unet_classifier = Model(inputs=input_layer, outputs=output_layer)
unet_classifier.summary()
# plot_model(unet_classifier, to_file=mplot, show_shapes=True, show_layer_names=True)

In [ ]:
# Compile
es_val_loss = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)
# es_val_accu = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.001)
mc_val_accu = ModelCheckpoint(str(model_max_accuracy.absolute()), monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
mc_val_loss = ModelCheckpoint(str(model_min_loss.absolute()), monitor='val_loss', mode='min', verbose=1, save_best_only=True)

tb = TensorBoard(
    log_dir=log_d, 
    histogram_freq=1, 
    write_graph=True, 
    write_images=True,
    update_freq='batch', 
    embeddings_freq=0,
    embeddings_metadata=None
)


unet_classifier.compile(
    optimizer=Adam(learning_rate=1e-4), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
train_generator = RasterDataGenerator( 
    map_dict=train_map,
    channels=bands,
    img_height=image_height,
    img_width=image_width,
    win_height=window_height,
    win_width=window_width,
    min_hoverlap=min_height_overlap,
    min_woverlap=min_width_overlap,
    cls_count=class_count,
    boundless=boundless_flag,
    shuffle=data_shuffle,
    batch_size=batchsize
)
valid_generator = RasterDataGenerator(
    map_dict=valid_map,
    channels=bands,
    img_height=image_height,
    img_width=image_width,
    win_height=window_height,
    win_width=window_width,
    min_hoverlap=min_height_overlap,
    min_woverlap=min_width_overlap,
    cls_count=class_count,
    boundless=boundless_flag,
    shuffle=data_shuffle,
    batch_size=batchsize
)

In [ ]:
# Train
unet_classifier.fit_generator(
    generator=train_generator, 
    epochs=50, 
    validation_data=valid_generator,
    use_multiprocessing=False,
    callbacks=[
        tb,
        es_val_loss,
        mc_val_loss,
#         es_val_accu,
        mc_val_accu,
        
    ]
)